In [19]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

import numpy as np

## Ссылки с открывающей страницы

In [14]:
url = 'https://outmaxshop.ru/snickers'

data = requests.get(url).text

soup_op = BeautifulSoup(data, 'html.parser')

links_op = soup_op.find('div', {'class': 'catalog-product__list'}).find_all('a')

links_op = [i['href'] for i in links_op]



## Функция, для получения ссылок на карточки товаров

In [8]:
def find_links(number):
    url = 'https://outmaxshop.ru/snickers?start=' + str(number)
    
    data = requests.get(url).text

    soup = BeautifulSoup(data, 'html.parser')
    
    a = soup.find('div', {'class': 'catalog-product__list'}).find_all('a')
    
    links = [i['href'] for i in a]
    
    return links

## Функция, для получения описания товара

In [9]:
def find_description(soup):
    b = soup.find('table', {'class': 'product-chars'})
    c = soup.find('table', {'class': 'product-chars'})
    if b is None or c is None:
        return [np.nan] * 5
    
    b = b.find_all('td', {'class': 'product-chars__text'})
    c = c.find_all('td', {'class': 'product-chars__name'}) 
    names = [(i.text).strip() for i in c]  
    desc = [(i.text).strip() for i in b]
    dic = {i: j for i,j in zip(names, desc)}
    info = []
    for i in ['Материал верха', 'Модель', 'Категория', 'Сезон', 'Цвет']:
        if i in dic:
            info.append(dic[i])
        else:
            info.append(np.nan)
    
    return info

## Функция для получения брэнда товара

In [10]:
def find_brand(soup):
    brand = soup.find('a', {'class': 'product__brand'})
    if brand is None:
        return [np.nan]
    else:
        brand = brand.find_all('meta')
        return [brand[0]['content']]

## Функция, для получения цен товара

In [11]:
def find_price(soup):
    prices_old = (soup.find_all('div', {'class': 'product__price product__price--line'}))[0].text
    prices_current = (soup.find_all('div', {'class': 'product__price product__price--red'}))[0].text

    price_old = int((prices_old.strip()).replace('руб.', '').replace(' ', ''))
    price_current = int((prices_current.strip()).replace('руб.', '').replace(' ', ''))
    
    prices = [price_old, price_current]
    
    return prices

##  Функция, для получения размеров товара

In [12]:
def find_sizes(soup):
    all_sizes = soup.find('div', {'class': 'product-size__container'}).find_all('span', 'radio_attr_label')
    
    fin_all_sizes = ','.join([i.text for i in all_sizes])

    return [fin_all_sizes]

## Функция для получения html-кода страницы

In [13]:
def get_the_soup(link):
    url = 'https://outmaxshop.ru' + link
    
    data = requests.get(url).text
    
    soup = BeautifulSoup(data, 'html.parser')
    
    return soup

In [ ]:
for i in links_op:
    print(i)
    try:
        soup = get_the_soup(i)
        
        brand = find_brand(soup)
        
        description = find_description(soup)
        
        
        prices = find_price(soup)
        
        sizes = find_sizes(soup)
        
        infor = brand + description + sizes + prices
        

        
        df.loc[len(df)] = infor
    except Exception:
        df.loc[len(df)] = [np.nan] * 9
        

In [24]:
for i in range(40,5090,40):
    needed_links = find_links(i)
    print(i)
    
    for j in needed_links:
        try:
            soup = get_the_soup(j)
        
            brand = find_brand(soup)
        
            description = find_description(soup)
        
        
            prices = find_price(soup)
        
            sizes = find_sizes(soup)
        
            infor = brand + description + sizes + prices
        
            df.loc[len(df)] = infor
        except Exception:
            df.loc[len(df)] = [np.nan] * 9